<a href="https://colab.research.google.com/github/tinumide/AlphaQuest/blob/develop/POC_X.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install transformers

In [ ]:
!pip install sacrebleu
!pip install evaluate

In [ ]:
!pip install rouge_score

In [ ]:
import os

import torch
import numpy as np
import evaluate


from datasets import load_dataset
from transformers import (
    AdamW,
    AutoTokenizer, 
    DataCollatorForSeq2Seq,
    get_scheduler,
    GPT2LMHeadModel,
    GPT2Tokenizer, 
)
from torch.utils.data import DataLoader, Dataset
from tqdm.auto import tqdm

device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")

In [ ]:
dataset = load_dataset("deepmind/code_contests")

In [ ]:
dataset = dataset.flatten()

In [ ]:
dataset = dataset.remove_columns(['name',
 'public_tests.input',
 'public_tests.output',
 'private_tests.input',
 'private_tests.output',
 'generated_tests.input',
 'generated_tests.output',
 'source',
 'difficulty',
 'solutions.language',
 'incorrect_solutions.language',
 'incorrect_solutions.solution',
 'cf_contest_id',
 'cf_index',
 'cf_points',
 'cf_rating',
 'cf_tags',
 'is_description_translated',
 'untranslated_description',
 'time_limit.seconds',
 'time_limit.nanos',
 'memory_limit_bytes',
 'input_file',
 'output_file'])


In [ ]:
dataset = dataset.filter(lambda example: len(example['solutions.solution']) > 0)

In [ ]:
def prep(example):
  if example["solutions.solution"][0]:
    example["solutions.solution"] = example["solutions.solution"][0]
  return example

dataset = dataset.map(prep)

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium', padding_side='left')

In [ ]:
def tokenize_data(example):
  return tokenizer(example["solutions.solution"], text_target=example['description'], max_length=500, truncation=True, padding=True)


In [ ]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenized_data = dataset.map(tokenize_data, batched=True)

In [ ]:
tokenized_data.set_format("torch")
tokenized_data["train"].column_names

['description', 'solutions.solution', 'input_ids', 'attention_mask', 'labels']

In [ ]:
tokenized_data = tokenized_data.remove_columns(["description", "solutions.solution"])

In [ ]:
model = GPT2LMHeadModel.from_pretrained('gpt2-medium')
model = model.to(device)
model.resize_token_embeddings(len(tokenizer))

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

train_dataloader = DataLoader(
    tokenized_data["train"], shuffle=True, batch_size=1, collate_fn=data_collator)
eval_dataloader = DataLoader(
    tokenized_data["valid"], batch_size=1, collate_fn=data_collator
)

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

{'input_ids': torch.Size([1, 500]),
 'attention_mask': torch.Size([1, 500]),
 'labels': torch.Size([1, 500])}

In [ ]:
optimizer = AdamW(model.parameters(), lr=5e-5)
num_epochs = 2
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
def postprocess(predictions, labels):
    predictions = predictions.cpu().numpy()
    labels = labels.cpu().numpy()

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    return decoded_preds, decoded_labels

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
progress_bar = tqdm(range(num_training_steps))
models_folder = '/content/sample_data'

model.train()

for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

torch.save(model.state_dict(), os.path.join(models_folder, f"alpha_quest.pt"))


  0%|          | 0/26268 [00:00<?, ?it/s]

In [ ]:
bleu_score = evaluate.load("sacrebleu")
rouge_score = evaluate.load("rouge")
model.eval()
for batch in tqdm(eval_dataloader):
    with torch.no_grad():
        generated_tokens = model.generate(
            batch["input_ids"].to(device),
            attention_mask=batch["attention_mask"].to(device),
            max_length=1024,
        )
    labels = batch["labels"]

    decoded_preds, decoded_labels = postprocess(generated_tokens, labels)
    bleu_score.add_batch(predictions=decoded_preds, references=decoded_labels)
    rouge_score.add_batch(predictions=decoded_preds, references=decoded_labels)

bleu_results = bleu_score.compute()
rouge_results = rouge_score.compute()

In [ ]:
print(f"BLEU score: {bleu_results['score']:.2f}")

BLEU score: 8.54


In [ ]:
print(f"ROUGE score: {rouge_results}")

ROUGE score: {'rouge1': 0.14555384550458877, 'rouge2': 0.03904931485646447, 'rougeL': 0.09847428187161715, 'rougeLsum': 0.1383524471724976}


In [ ]:
rouge_score = evaluate.load("rouge")

In [ ]:
tokenized_data["test"][:2]

In [ ]:
model_path = os.path.join(models_folder, f"alpha_quest.pt")
model.load_state_dict(torch.load(model_path))

prompts = tokenized_data["test"][:5]

sample_outputs = model.generate(prompts["input_ids"].to(device), 
                                attention_mask=prompts["attention_mask"].to(device), 
                                max_length=1024
                                )


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
for i, output in enumerate(sample_outputs ):
    print("{}: {}".format(i, tokenizer.decode(output, skip_special_tokens=True)))

# Generated Sequences

1. 

```
#include <bits/stdc++.h>
using namespace std;
bool isrange(int second, int first, int n, int m) {
  if (0 <= second && second < n && 0 <= first && first < m) return true;
  return false;
}
int dy[4] = {1, 0, -1, 0}, dx[4] = {0, 1, 0, -1},
    ddy[8] = {1, 0, -1, 0, 1, 1, -1, -1}, ddx[8] = {0, 1, 0, -1, 1, -1, 1, -1};
const int MAX = 1e6 + 100;
int n, m;
pair<string, int> s[MAX];
bool comp(pair<string, int>& a, pair<string, int>& b) {
  for (int e = 0; e < m; e++) {
    if (e % 2 == 0) {
      if (a.first[e]!= b.first[e]) return a.first[e] < b.first[e];
    } else {
      if (a.first[e]!= b.first[e]) return a.first[e] > b.first[e];
    }
  }
  return true;
}
void solve(int tc) {
  cin >> n >> m;
  for (int e = 0; e < n; e++) {
    cin >> s[e].first;
    s[e].second = e + 1;
  }
  sort(s, s + n, comp);
  for (int e = 0; e < n; e++) cout << s[e].second << " ";
}
int main(void) {
  ios_base ::sync_with_stdio(false);
  cin.tie(NULL);
  cout.tie(NULL);
  int tc = 1;
  for (int test_number = 1; test_number <= tc; test_number++) {
```

 2.

```
#include <bits/stdc++.h>
const double PI = 3.1415926535897932384626433;
const int KL = 3e5 + 10;
const long long MOD = 1e9 + 7;
using namespace std;
struct point {
  long double x, y;
  void go(long long x1, long long y1) {
    x = x1;
    y = y1;
  }
  void read() { cin >> x >> y; }
  point operator-(point b) { return point{x - b.x, y - b.y}; }
  point operator+(point b) { return point{x + b.x, y + b.y}; }
};
long double fnd_angle(point p) {
  long double len = sqrt(p.x * p.x + p.y * p.y);
  long double angle = acos(fabs(p.x) / len);
  if (p.y < 0) {
    if (p.x >= 0)
      angle = 2 * PI - angle;
    else
      angle += PI;
  } else if (p.x < 0)
    angle = PI - angle;
  return angle;
}
int n, k;
vector<point> A;
vector<pair<long double, long double>> a, b;
multiset<long double> s;
bool ok(long double mid) {
  a.clear();
  b.clear();
  for (int i = 0; i < n; i++) {
    long double dis = sqrt(A[i].x * A[i].x + A[i].y * A[i].y);
    if (dis > 2 * mid) continue;
    long double angle = fnd_angle(A[i]);
    long double angle2 = acos(dis / (2.0 * mid));
    a.push_back({angle - angle2, 2 * angle2});
    b.push_back
```
3.

```
#include <bits/stdc++.h>
using namespace std;
struct comp {
  double a, b;
  comp() {}
  comp(double x) : a(x), b(0) {}
  comp(double x, double y) : a(x), b(y) {}
};
inline comp operator+(comp a, comp b) { return comp(a.a + b.a, a.b + b.b); }
inline comp operator-(comp a, comp b) { return comp(a.a - b.a, a.b - b.b); }
inline comp operator*(comp a, comp b) {
  return comp(a.a * b.a - a.b * b.b, a.a * b.b + a.b * b.a);
}
ostream& operator<<(ostream& a, comp b) {
  a << "(" << b.a << "," << b.b << ")";
  return a;
}
int K;
comp w[2][1234567];
comp qp(comp a, long long b) {
  comp x = 1;
  while (b) {
    if (b & 1) x = x * a;
    a = a * a;
    b >>= 1;
  }
  return x;
}
long long qp(long long a, long long b, int MOD) {
  long long x = 1;
  a %= MOD;
  while (b) {
    if (b & 1) x = x * a % MOD;
    a = a * a % MOD;
    b >>= 1;
  }
  return x;
}
const double pi = acos(-1);
int rev[1234567];
void init(int s) {
  for (K = 1; K < s; K <<= 1)
    ;
  {
    double t = pi * 2 / K;
    for (int i = 0; i <= K; ++i) w[0][i] = comp(cos(t
 [PAD] 
```

4.



```
s = input()
ways = 0
if len(s) == 1:
    if s in ['_', 'X', '0']:
        ways = 1
elif len(s) == 2:
    if s in ['25', '50', '75']:
        ways = 1
    elif s in ['__', '_X', 'X_']:
        ways = 3
    elif s == '_0':
        ways = 1
    elif s in ['_5', 'X5']:
        ways = 2
    elif s in ['2_', '5_', '7_', '2X', '5X', '7X', 'X0']:
        ways = 1
else:
    d = s[-2:]
    s = s[:-2]
    if d in ['00', '25', '50', '75', '__', '_X', 'X_', 'XX', '_0', '_5', '0_', '2_', '5_', '7_', 'X0', 'X5', '0X', '2X', '5X', '7X']:
        ways = 1
        xfound = False

        if d in ['__', '_X', 'X_']:
            ways *= 4
        elif d in ['_0', '_5', 'X0', 'X5']:
            ways *= 2
        
        if 'X' in d:
            xfound = True
        
        if s[0] == '_' or (s[0] ==
```


5



```
#include <bits/stdc++.h>
#pragma GCC optimize(2, 3, "Ofast")
using namespace std;
template <typename T1, typename T2>
void ckmin(T1 &a, T2 b) {
  if (a > b) a = b;
}
template <typename T1, typename T2>
void ckmax(T1 &a, T2 b) {
  if (a < b) a = b;
}
int read() {
  int x = 0, f = 0;
  char ch = getchar();
  while (!isdigit(ch)) f |= ch == '-', ch = getchar();
  while (isdigit(ch)) x = 10 * x + ch - '0', ch = getchar();
  return f? -x : x;
}
template <typename T>
void print(T x) {
  if (x < 0) putchar('-'), x = -x;
  if (x >= 10) print(x / 10);
  putchar(x % 10 + '0');
}
template <typename T>
void print(T x, char let) {
  print(x), putchar(let);
}
const int N = 200005;
const int mod = 1e9 + 7;
const int inf = 1e9;
inline void add(int &x, int y) {
  x += y;
  if (x >= mod) x -= mod;
}
inline void sub(int &x, int y) {
  x -= y;
  if (x < 0) x += mod;
}
vector<pair<int, int> > adj[N];
int a[N], n, LIM;
bool ban[N];
int sz[N], maxp[N], sumsz, rt;
void getrt(int u, int fa) {
  maxp[u] = 0, sz[u] = 1;
  for (auto v : adj[u]) {
    if (ban[v.first] || v.first ==

```










In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp "/content/sample_data/alpha_quest.pt" "/content/drive/My Drive"